<a href="https://colab.research.google.com/github/protovici/PyTorch_Lightning_Dev/blob/main/lightning_cnn_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PyTorch Lightning Dev | page 38 Start

In [44]:
!pip install torch==1.10.0 torchvision==0.11.1 torchtext==0.11.0 torchaudio==0.10.0 --quiet
!pip install pytorch-lightning==1.5.2 --quiet
!pip install seaborn==0.11.2
!pip install numpy
!pip install pandas==1.3.5
!pip install opendatasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
import os
import shutil
import opendatasets as od
import pandas as pd
import numpy as np
from PIL import Image
from google.colab import drive
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
from torchvision.utils import make_grid
from torchmetrics.functional import accuracy
import pytorch_lightning as pl
from google.colab import drive

In [7]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
dataset_url = 'https://www.kaggle.com/competitions/histopathologic-cancer-detection/data'
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: protovici
Your Kaggle Key: ··········


100%|██████████| 6.31G/6.31G [00:50<00:00, 134MB/s]



Extracting archive ./histopathologic-cancer-detection/histopathologic-cancer-detection.zip to ./histopathologic-cancer-detection


In [46]:
cancer_labels = pd.read_csv('/content/histopathologic-cancer-detection/train_labels.csv')
cancer_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220025 entries, 0 to 220024
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      220025 non-null  object
 1   label   220025 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.4+ MB


In [47]:
APP_FOLDER = '/content/histopathologic-cancer-detection'
totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(APP_FOLDER):
    print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        totalFiles += 1

print('Total number of files',totalFiles)
print('Total Number of directories',totalDir)
print('Total:',(totalDir + totalFiles))

Searching in :  /content/histopathologic-cancer-detection
Searching in :  /content/histopathologic-cancer-detection/test
Searching in :  /content/histopathologic-cancer-detection/train
Searching in :  /content/histopathologic-cancer-detection/train_dataset
Searching in :  /content/histopathologic-cancer-detection/test_dataset
Total number of files 287321
Total Number of directories 4
Total: 287325


In [48]:
np.random.seed(0)

train_imgs_orig = os.listdir("/content/histopathologic-cancer-detection/train")

selected_image_list = []

for img in np.random.choice(train_imgs_orig, 10000):
 selected_image_list.append(img)
len(selected_image_list)

10000

In [49]:
np.random.seed(0)
np.random.shuffle(selected_image_list)

cancer_train_idx = selected_image_list[:8000]
cancer_test_idx = selected_image_list[8000:]

print("Number of images in the downsampled training dataset: ", len(cancer_train_idx))
print("Number of images in the downsampled testing dataset: ", len(cancer_test_idx))

Number of images in the downsampled training dataset:  8000
Number of images in the downsampled testing dataset:  2000


In [50]:
#os.mkdir('/content/histopathologic-cancer-detection/train_dataset/')

for fname in cancer_train_idx:
  src = os.path.join('histopathologic-cancer-detection/train', fname)
  dst = os.path.join('/content/histopathologic-cancer-detection/train_dataset/', fname)
  shutil.copyfile(src, dst)

print('No. of images in downsampled training dataset: ', len(os.listdir("/content/histopathologic-cancer-detection/train_dataset/")))

No. of images in downsampled training dataset:  7847


In [52]:
#os.mkdir('/content/histopathologic-cancer-detection/test_dataset/')

for fname in cancer_test_idx:
  src = os.path.join('histopathologic-cancer-detection/train', fname)
  dst = os.path.join('/content/histopathologic-cancer-detection/test_dataset/', fname)
  shutil.copyfile(src, dst)

print('No. of images in downsampled testing dataset: ', len(os.listdir("/content/histopathologic-cancer-detection/test_dataset/")))

No. of images in downsampled testing dataset:  1989


In [53]:
selected_image_labels = pd.DataFrame()

id_list = []
label_list = []

for img in selected_image_list:
  label_tuple = cancer_labels.loc[cancer_labels['id'] == img.split('.')[0]]
  id_list.append(label_tuple['id'].values[0])
  label_list.append(label_tuple['label'].values[0])

In [54]:
class LoadCancerDataset(Dataset):
  def __init__(self, data_folder, transform = T.Compose([T.CenterCrop(32),T.ToTensor()]), dict_labels={}):
    self.data_folder = data_folder
    self.list_image_files = [s for s in os.listdir(data_folder)]
    self.transform = transform
    self.dict_labels = dict_labels
    self.labels = [dict_labels[i.split('.')[0]] for i in self.list_image_files]

  def __len__(self):
    return len(self.list_image_files)

  def __getitem__(self, idx):
    img_name = os.path.join(self.data_folder, self.list_image_files[idx])
    image = Image.open(img_name)
    image = self.transform(image)
    img_name_short = self.list_image_files[idx].split('.')[0]
    label = self.dict_labels[img_name_short]
    return image, label

### Below code needs debugged.

- [Link to current book location](https://subscription.packtpub.com/book/data/9781800561618/3/ch03lvl1sec14/building-a-cnn-model-for-image-recognition)

In [55]:
img_class_dict = {k:v for k, v in zip(selected_image_labels.id, selected_image_labels.label)}

AttributeError: ignored